<a href="https://colab.research.google.com/github/ihrisikesa/DWS/blob/main/DWS_Sahabat_Sehat_Progress_Lobar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- installs (first run only) ---
!pip -q install gspread gspread_dataframe google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas==2.2.2

# --- auth ---
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import google.auth
from googleapiclient.discovery import build
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# --- credentials and services ---
creds, _ = google.auth.default()
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/documents",
]
creds = creds.with_scopes(SCOPES)

gc = gspread.authorize(creds)
drive = build("drive", "v3", credentials=creds)
docs  = build("docs",  "v1", credentials=creds)

In [2]:
# ==== 0) Setup & imports ====
from google.colab import drive
drive.mount('/content/drive')  # skip if already mounted

from pathlib import Path
import pandas as pd
import numpy as np
import re

# ==== 1) Config ====
FOLDER = Path('/content/drive/MyDrive/sahabat_sehat_lobar/dws/DWS export (File responses)/File (File responses)')
FILE_PATTERN = '*.csv'   # adjust if needed
THRESH_MIN = 3.788                              # threshold in minutes
FILTER_KUNJUNGAN_SUCCESS = False                # set True to keep only Kunjungan Rumah + Success

# ==== 2) Read latest matching CSV from folder ====
matches = list(FOLDER.glob(FILE_PATTERN))
if not matches:
    raise FileNotFoundError(f"No matching CSV found in: {FOLDER} with pattern {FILE_PATTERN}")

latest = max(matches, key=lambda p: p.stat().st_mtime)
print(f"Reading: {latest.name}")

# Let pandas infer delimiter robustly (comma/semicolon)
df = pd.read_csv(latest, encoding='utf-8-sig', sep=None, engine='python')

# ==== 3) Find key columns robustly ====
def find_col(candidates_regex, columns):
    """
    Return the first column whose name matches any regex in candidates_regex (case-insensitive).
    """
    for pat in candidates_regex:
        for c in columns:
            if re.search(pat, str(c), flags=re.I):
                return c
    return None

manual_col = find_col(
    [r'^manual\s*check[- ]?in$', r'manual\s*check[- ]?in'], df.columns
)
if manual_col is None:
    raise KeyError("Couldn't find a 'Manual check-in' column. "
                   f"Available columns: {list(df.columns)}")

date_col = find_col(
    [r'^date$', r'^tanggal$', r'^tgl$'], df.columns
)
if date_col is None:
    raise KeyError("Couldn't find a Date/Tanggal column. "
                   "Expected one of: Date, Tanggal, tgl (case-insensitive).")

print(f"Using columns -> Manual check-in: '{manual_col}', Date: '{date_col}'")

# ==== 4) Parse 'Manual check-in' durations to minutes ====
def minutes_from_manual(series: pd.Series) -> pd.Series:
    """
    Parse durations like:
      '2 h 10 m', '15 m', '900 s', '1h', '1h30m', '75m 30s', '12:30', or plain '15'
    Returns minutes as float.
    """
    s = series.astype("string")

    def _to_min(x) -> float:
        if x is None or pd.isna(x):
            return np.nan
        x = str(x).strip().lower()
        if not x:
            return np.nan

        # capture number+unit pairs (h/m/s), with or without spaces (e.g., '1h30m', '2 h 10 m')
        pairs = re.findall(r'(\d+(?:\.\d+)?)\s*([hms])', x)
        if pairs:
            total = 0.0
            for val, unit in pairs:
                v = float(val)
                if unit == 'h':
                    total += v * 60
                elif unit == 'm':
                    total += v
                elif unit == 's':
                    total += v / 60.0
            return total

        # handle mm:ss format
        if re.fullmatch(r'\d+:\d{1,2}', x):
            mm, ss = x.split(':')
            return float(mm) + float(ss) / 60.0

        # plain number -> assume minutes
        if re.fullmatch(r'\d+(?:\.\d+)?', x):
            return float(x)

        return np.nan

    return s.map(_to_min).astype(float)

df["dur_ci_co_min"] = minutes_from_manual(df[manual_col])


# ==== 5) Optional filter to Kunjungan Rumah + (Success OR Scheduled) ====
if FILTER_KUNJUNGAN_SUCCESS:
    df_kunj = df.loc[
        df["Title"].eq("Kunjungan Rumah")
        & df["Status"].isin(["Success"])
    ].copy()
else:
    df_kunj = df.copy()


# ==== 6) Build Date key (date only) ====
df_kunj["Date_key"] = pd.to_datetime(df_kunj[date_col], errors="coerce").dt.date

# ==== 7) Indicators vs 3.788 minutes ====
mins = pd.to_numeric(df_kunj["dur_ci_co_min"], errors="coerce")
df_kunj["lt_3_788"] = (mins < THRESH_MIN).astype("Int8")
df_kunj["gt_3_788"] = (mins > THRESH_MIN).astype("Int8")

# ==== 8) Group & aggregate ====
# Group by Worker × Date × Title × Status; keep NaNs (dropna=False)
gb = (df_kunj.groupby(["Worker", "Date_key", "Title", "Status"], dropna=False, as_index=False)
      .agg(
          total_m      = ("dur_ci_co_min", lambda s: pd.to_numeric(s, errors="coerce").fillna(0).sum()),
          n_rows       = ("dur_ci_co_min", "size"),
          n_nonnull    = ("dur_ci_co_min", lambda s: pd.to_numeric(s, errors="coerce").notna().count()),
          lt_3_788_sum = ("lt_3_788", "sum"),
          gt_3_788_sum = ("gt_3_788", "sum"),
      ))

# Cast numeric columns safely
for c in ["total_m", "n_rows", "n_nonnull", "lt_3_788_sum", "gt_3_788_sum"]:
    gb[c] = pd.to_numeric(gb[c], errors="coerce").fillna(0).astype("int64")

# Averages & percentages (use n_nonnull to avoid bias from missing durations)
gb["avg_m"] = np.where(gb["n_nonnull"] > 0, gb["total_m"] / gb["n_nonnull"], np.nan).round(2)
gb["percentage_<3788"] = np.where(gb["n_nonnull"] > 0, gb["lt_3_788_sum"] / gb["n_nonnull"] * 100, np.nan).round(2)
gb["percentage_>3788"] = np.where(gb["n_nonnull"] > 0, gb["gt_3_788_sum"] / gb["n_nonnull"] * 100, np.nan).round(2)

# ==== 9) Tidy & rename ====
gb = (gb.rename(columns={"Date_key":"Date"})
        .sort_values(["Worker","Date","Title","Status"])
        .rename(columns={
            "total_m": "total_duration_minutes",
            "n_rows":  "n_visit",
            "avg_m":   "avg_duration_minutes",
            "lt_3_788_sum": "<3788_sum",
            "gt_3_788_sum": ">3788_sum"
        }))

# Optional: keep only Kunjungan Rumah + Success in the final table (independent of earlier toggle)
gb_kunj = gb[(gb["Title"] == "Kunjungan Rumah") & (gb["Status"].isin(["Success"]))].copy()

# ==== 10) Peek results ====
print("\n=== Aggregated (all titles/status) head ===")
display(gb.head(10))

print("\n=== Kunjungan Rumah + Success head ===")
display(gb_kunj.head(10))

Mounted at /content/drive
Reading: jobs-report (2) - Septhia Salwa Zulfatiha.csv
Using columns -> Manual check-in: 'Manual check-in', Date: 'Date'

=== Aggregated (all titles/status) head ===


,Worker,Date,Title,Status,total_duration_minutes,n_visit,n_nonnull,<3788_sum,>3788_sum,avg_duration_minutes,percentage_<3788,percentage_>3788
0,Baiq Liza Indriyani,2025-10-06,Kunjungan Rumah,Scheduled,0,57,57,0,0,0.00,0.00,0.00
1,Baiq Liza Indriyani,2025-10-06,Kunjungan Rumah,Success,7,13,13,13,0,0.54,100.00,0.00
2,Baiq Liza Indriyani,2025-10-06,Revisit Kunjungan Rumah,Scheduled,0,1,1,0,0,0.00,0.00,0.00
3,Dewi Anggraeni,2025-10-06,Kunjungan Rumah,Scheduled,0,10,10,0,0,0.00,0.00,0.00
4,Dewi Anggraeni,2025-10-06,Kunjungan Rumah,Success,143,60,60,49,10,2.38,81.67,16.67
5,Erni Siswati,2025-10-06,Kunjungan Rumah,Scheduled,0,12,12,0,0,0.00,0.00,0.00
6,Erni Siswati,2025-10-06,Kunjungan Rumah,Started,0,1,1,0,0,0.00,0.00,0.00
7,Erni Siswati,2025-10-06,Kunjungan Rumah,Success,114,53,53,41,8,2.15,77.36,15.09
8,Erni Siswati,2025-10-06,Revisit Kunjungan Rumah,Issue,0,1,1,1,0,0.00,100.00,0.00
9,Erni Siswati,2025-10-06,Revisit Kunjungan Rumah,Success,28,3,3,1,2,9.33,33.33,66.67



=== Kunjungan Rumah + Success head ===


,Worker,Date,Title,Status,total_duration_minutes,n_visit,n_nonnull,<3788_sum,>3788_sum,avg_duration_minutes,percentage_<3788,percentage_>3788
1,Baiq Liza Indriyani,2025-10-06,Kunjungan Rumah,Success,7,13,13,13,0,0.54,100.00,0.00
4,Dewi Anggraeni,2025-10-06,Kunjungan Rumah,Success,143,60,60,49,10,2.38,81.67,16.67
7,Erni Siswati,2025-10-06,Kunjungan Rumah,Success,114,53,53,41,8,2.15,77.36,15.09
11,Fitria Rohayani,2025-10-06,Kunjungan Rumah,Success,88,51,51,42,7,1.73,82.35,13.73
13,Hayatun,2025-10-06,Kunjungan Rumah,Success,130,74,74,70,2,1.76,94.59,2.70
17,Indana Ajmala Tifani,2025-10-06,Kunjungan Rumah,Success,146,49,49,35,14,2.98,71.43,28.57
20,Marlia Septiana,2025-10-06,Kunjungan Rumah,Success,133,62,62,55,7,2.15,88.71,11.29
24,Mira Zalila,2025-10-06,Kunjungan Rumah,Success,86,46,46,41,5,1.87,89.13,10.87
27,Muhibbah,2025-10-06,Kunjungan Rumah,Success,4,56,56,56,0,0.07,100.00,0.00
29,Muslimah,2025-10-06,Kunjungan Rumah,Success,63,49,49,46,3,1.29,93.88,6.12


In [3]:
# Wide table of status counts per Worker × Date × Title
status_wide = (gb[["Worker", "Date", "Title", "Status", "n_visit"]]
               .assign(Status=lambda d: d["Status"].astype("string").str.strip())   # optional normalize
               .pivot_table(index=["Worker", "Date", "Title"],
                            columns="Status",
                            values="n_visit",
                            aggfunc="sum",
                            fill_value=0)
               .reset_index())

# Drop the columns axis name ("Status")
status_wide.columns.name = None

# Ensure these columns exist even if absent in data
for c in ["Success", "Scheduled", "Issues", "Started"]:
    if c not in status_wide.columns:
        status_wide[c] = 0

# Total across the four statuses
status_cols = ["Success", "Scheduled", "Issues", "Started"]
status_wide[status_cols] = status_wide[status_cols].astype("int64")
status_wide["total_jobs"] = status_wide[status_cols].sum(axis=1).astype("int64")

# % Success per total jobs (0–100). Safe for zero totals.
status_wide["pct_success"] = np.where(
    status_wide["total_jobs"] > 0,
    status_wide["Success"] / status_wide["total_jobs"] * 100,
    np.nan
).round(2)

# (optional) pretty text like "83.33%"
status_wide["pct_success_str"] = status_wide["pct_success"].map(
    lambda x: f"{x:.2f}%" if pd.notna(x) else ""
)

# Reorder and (optionally) filter to Kunjungan Rumah
status_wide = status_wide[["Worker", "Date", "Title"] + status_cols + ["total_jobs", "pct_success", "pct_success_str"]]
status_wide = status_wide.query("Title == 'Kunjungan Rumah'").copy()

status_wide.head(10)
status_wide.columns


Index(['Worker', 'Date', 'Title', 'Success', 'Scheduled', 'Issues', 'Started',
       'total_jobs', 'pct_success', 'pct_success_str'],
      dtype='string')

In [4]:
# Assume you already have:
# gb_kunj  (from your grouped Kunjungan Rumah table)
# status_wide (wide counts with Success/Scheduled/Issues/Started/total_jobs/pct_success)

import pandas as pd

# 1) Make sure the join keys have the same types/format
gb_kunj["Worker"] = gb_kunj["Worker"].astype("string").str.strip()
status_wide["Worker"] = status_wide["Worker"].astype("string").str.strip()

gb_kunj["Title"] = gb_kunj["Title"].astype("string").str.strip()
status_wide["Title"] = status_wide["Title"].astype("string").str.strip()

gb_kunj["Date"] = pd.to_datetime(gb_kunj["Date"], errors="coerce").dt.date
status_wide["Date"] = pd.to_datetime(status_wide["Date"], errors="coerce").dt.date

# 2) (Optional) sanity check: status_wide should be unique on the keys
# If not unique, uncomment to compress to one row
# if status_wide.duplicated(["Worker","Date","Title"]).any():
#     status_cols = ["Success","Scheduled","Issues","Started","total_jobs"]
#     status_wide = (status_wide
#                    .groupby(["Worker","Date","Title"], as_index=False)[status_cols].sum())

# 3) Join (left join to keep all gb_kunj rows)
final_tbl = gb_kunj.merge(
    status_wide,
    on=["Worker", "Date", "Title"],
    how="left",
    validate="m:1"   # many gb_kunj rows to 1 status_wide row
)

# 4) Clean up missing values & types for the added status columns
for c in ["Success", "Scheduled", "Issues", "Started", "total_jobs"]:
    if c in final_tbl.columns:
        final_tbl[c] = pd.to_numeric(final_tbl[c], errors="coerce").fillna(0).astype("int64")

if "pct_success" in final_tbl.columns:
    final_tbl["pct_success"] = pd.to_numeric(final_tbl["pct_success"], errors="coerce").round(2)

if "pct_success_str" in final_tbl.columns:
    final_tbl["pct_success_str"] = final_tbl["pct_success_str"].fillna("")

# 5) Optional: reorder columns for readability
order = [
    "Worker","Date","Title","Status",
    "total_duration_minutes","n_visit","n_nonnull","<3788_sum",">3788_sum",
    "avg_duration_minutes","percentage_<3788","percentage_>3788",
    "Success","Scheduled","Issues","Started","total_jobs","pct_success","pct_success_str"
]
final_tbl = final_tbl[[c for c in order if c in final_tbl.columns] +
                      [c for c in final_tbl.columns if c not in order]]

# Preview
final_tbl.head(10)
final_tbl.columns

Index(['Worker', 'Date', 'Title', 'Status', 'total_duration_minutes',
       'n_visit', 'n_nonnull', '<3788_sum', '>3788_sum',
       'avg_duration_minutes', 'percentage_<3788', 'percentage_>3788',
       'Success', 'Scheduled', 'Issues', 'Started', 'total_jobs',
       'pct_success', 'pct_success_str'],
      dtype='object')

In [5]:
#!pip install gspread gspread_dataframe
#from google.colab import auth
#auth.authenticate_user()



In [6]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default

creds, project = default()
gc = gspread.Client(auth=creds)

sh = gc.open_by_key("1pInt5qeyZuvvqYuEm7ckhF7iIxnAGkXvdZIAWriHCJk") # SHEET_ID of DWS summary report sheet

try:
    ws = sh.worksheet("Summary") #sheet name
except gspread.exceptions.WorksheetNotFound:
    # First time: create + write everything (with header)
    ws = sh.add_worksheet(title="Summary",
                          rows=str(len(final_tbl)+10),
                          cols=str(len(final_tbl.columns)+5))
    set_with_dataframe(ws, final_tbl, include_index=False,
                       include_column_header=True, resize=True)
    print("Created 'Summary' and wrote initial data.")
else:
    # Read existing (use row 1 as header). Drop empty rows/cols.
    existing = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    existing = (existing.dropna(how="all")
                        .loc[:, ~existing.columns.str.contains(r"^Unnamed")])

    # Choose what to append:
    df_to_append = final_tbl.copy()

    # OPTIONAL: append only rows with a new date (adjust column name if needed)
    for date_col in ["Date", "Tanggal", "date", "tgl"]:
        if date_col in existing.columns and date_col in final_tbl.columns:
            ex_dates  = pd.to_datetime(existing[date_col], errors="coerce").dt.date
            new_dates = pd.to_datetime(final_tbl[date_col], errors="coerce").dt.date
            df_to_append = final_tbl[~new_dates.isin(set(ex_dates.dropna()))].copy()
            break

    if df_to_append.empty:
        print("Nothing new to append.")
    else:
        # First empty row (after header + data)
        start_row = len(ws.get_all_values()) + 1

        # Ensure the sheet has enough rows
        needed = start_row + len(df_to_append) - 1
        if needed > ws.row_count:
            ws.add_rows(needed - ws.row_count)

        # Append WITHOUT header and WITHOUT resizing
        set_with_dataframe(ws, df_to_append,
                           row=start_row, col=1,
                           include_index=False,
                           include_column_header=False,
                           resize=False)
        print(f"Appended {len(df_to_append)} rows to 'Summary'.")


Appended 25 rows to 'Summary'.


In [7]:
# =======================
# AUTH (single source of truth) – keep once
# =======================
from google.colab import auth
auth.authenticate_user()

import google.auth
from googleapiclient.discovery import build
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
import numpy as np

# Scopes + creds
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/documents",
]
creds, _ = default()
creds = creds.with_scopes(SCOPES)

# Clients (rename Drive service to avoid collision with colab drive)
gc         = gspread.authorize(creds)
drive_svc  = build("drive", "v3", credentials=creds)
docs_svc   = build("docs",  "v1", credentials=creds)

# If you need to mount the Colab Drive filesystem:
from google.colab import drive as gdrive
gdrive.mount('/content/drive')

# =======================
# CONFIG
# =======================
SHEET_ID          = "1pInt5qeyZuvvqYuEm7ckhF7iIxnAGkXvdZIAWriHCJk"
SHEET_TAB         = "Summary"
TEMPLATE_DOC_ID   = "1W-r9DzptEhUWpikAnhU9YDxc_QHcaQAYUKe-Y77fAcY"
DEST_FOLDER_ID    = None     # e.g. "your_folder_id"
OUTPUT_TAB        = "kartu_evaluasi_links"
SKIP_IF_HAS_LINK  = True     # set to False to force regeneration for testing

# =======================
# Helpers
# =======================
MONTH_ID = ["Januari","Februari","Maret","April","Mei","Juni",
            "Juli","Agustus","September","Oktober","November","Desember"]

def tanggal_id(dt: pd.Timestamp) -> str:
    dt = pd.to_datetime(dt, errors="coerce")
    if pd.isna(dt):
        return ""
    return f"{dt.day} {MONTH_ID[dt.month-1]} {dt.year}"

def copy_template(new_name: str) -> str:
    body = {"name": new_name}
    if DEST_FOLDER_ID:
        body["parents"] = [DEST_FOLDER_ID]
    file = drive_svc.files().copy(fileId=TEMPLATE_DOC_ID, body=body).execute()
    return file["id"]

def replace_placeholders(doc_id: str, mapping: dict):
    requests = [{"replaceAllText": {
        "containsText": {"text": f"{{{{{k}}}}}", "matchCase": True},
        "replaceText": str(v) if v is not None else ""
    }} for k, v in mapping.items()]
    docs_svc.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

# Quick permission smoke test – will raise if template not accessible
_ = docs_svc.documents().get(documentId=TEMPLATE_DOC_ID).execute()
print("Template is accessible.")

# =======================
# CONFIG
# =======================
SUCCESS_PCT_THRESHOLD = 60.0  # only create cards when %Success < 40

# =======================
# Read the source Sheet (keep all columns for metrics)
# =======================
ws = gc.open_by_key(SHEET_ID).worksheet(SHEET_TAB)
df_full = get_as_dataframe(ws, evaluate_formulas=True).dropna(how="all")

# normalize types
df_full["Worker"] = df_full.get("Worker").astype("string").str.strip()
df_full["Title"]  = df_full.get("Title").astype("string").str.strip()
df_full["Date"]   = pd.to_datetime(df_full.get("Date"), errors="coerce")

# numeric metrics
for col in ["Success", "total_jobs"]:
    if col in df_full.columns:
        df_full[col] = pd.to_numeric(df_full[col], errors="coerce")

# -----------------------
# Build metrics per Worker × Title × Date
# -----------------------
metrics = (df_full.dropna(subset=["Worker","Title","Date"])
                   .assign(Date=df_full["Date"].dt.date)
                   .groupby(["Worker","Title","Date"], as_index=False)
                   .agg(Success=("Success","sum"),
                        total_jobs=("total_jobs","sum")))

metrics["pct_success_calc"] = np.where(
    metrics["total_jobs"] > 0,
    metrics["Success"] / metrics["total_jobs"] * 100.0,
    np.nan
)

# -----------------------
# Filter to below-threshold combos
# -----------------------
to_generate = metrics.loc[
    (metrics["total_jobs"] > 0) & (metrics["pct_success_calc"] < SUCCESS_PCT_THRESHOLD)
].copy()

# Existing links (for skip)
try:
    ws_links = gc.open_by_key(SHEET_ID).worksheet(OUTPUT_TAB)
    existing_links = get_as_dataframe(ws_links).dropna(how="all")
except Exception:
    ws_links = None
    existing_links = pd.DataFrame(columns=["Worker","Title","Date","doc_url"])

if SKIP_IF_HAS_LINK and not existing_links.empty:
    ex = (existing_links[["Worker","Title","Date"]]
          .assign(Date=pd.to_datetime(existing_links["Date"], errors="coerce").dt.date))
    to_generate = (to_generate.merge(ex, on=["Worker","Title","Date"], how="left", indicator=True)
                             .loc[lambda d: d["_merge"].eq("left_only"),
                                  ["Worker","Title","Date","Success","total_jobs","pct_success_calc"]])

print("Rows to generate (< 60% success):", len(to_generate))
print(to_generate.head(10))

# =======================
# Create Docs for the filtered rows
# =======================
created = []
for r in to_generate.itertuples(index=False):
    worker, title, d, succ, tot, pct = r.Worker, r.Title, pd.to_datetime(r.Date), int(r.Success or 0), int(r.total_jobs or 0), float(r.pct_success_calc)

    nice_date = tanggal_id(d)
    doc_name  = f"Kartu Evaluasi - {worker} - {title} - {nice_date}"

    new_doc_id = copy_template(doc_name)
    temuan_str = f"{succ}/{tot} ({pct:.2f}%)"  # what goes into {{TEMUAN}}

    replace_placeholders(new_doc_id, {
        "NAMA": worker,
        "JENIS_KUNJUNGAN": title,
        "TANGGAL_KUNJUNGAN": nice_date,
        "TEMUAN": temuan_str,
        # optional extras if you added these placeholders:
        "SUKSES": succ,
        "TOTAL_JOBS": tot,
        "PCT_SUKSES": f"{pct:.2f}% Jobs"
    })

    created.append({
        "Worker": worker,
        "Title":  title,
        "Date":   d.date(),
        "doc_url": f"https://docs.google.com/document/d/{new_doc_id}/edit"
    })

links_df = pd.DataFrame(created)
print(f"Created {len(links_df)} documents (< {SUCCESS_PCT_THRESHOLD}% success).")

# =======================
# Write/append links tab
# =======================
sh = gc.open_by_key(SHEET_ID)
if ws_links is None:
    ws_links = sh.add_worksheet(title=OUTPUT_TAB, rows=200, cols=6)
    set_with_dataframe(ws_links, links_df, include_index=False, resize=True)
else:
    existing = get_as_dataframe(ws_links).dropna(how="all")
    out = pd.concat([existing, links_df], ignore_index=True)
    ws_links.clear()
    set_with_dataframe(ws_links, out, include_index=False, resize=True)

print("Links updated:", OUTPUT_TAB)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Template is accessible.
Rows to generate (< 60% success): 2
                 Worker            Title        Date  Success  total_jobs  \
3   Baiq Liza Indriyani  Kunjungan Rumah  2025-10-06       13          70   
20    Ni Ketut Suariani  Kunjungan Rumah  2025-10-06       33          67   

    pct_success_calc  
3          18.571429  
20         49.253731  
Created 2 documents (< 60.0% success).
Links updated: kartu_evaluasi_links
